# Trabalho disciplina de Introdução à Teoria da Computação
### Geração de Comentários Falsos do Mercado Livre utilizando Cadeias de Markov

Jonathan Daniel Ramos - 10857082

Daniel de Jesus Lima - 10723951

Matheus Aquati Kurianski - 10687541

Primeiramente, é necessário que limpemos nossos dados:

In [1]:
import re
import numpy as np

with open('comments.txt') as f:
    comments = f.read()

pontuations = ['.','-',',','!','?','(','—',')']
first_words = []
end_words = []

# Aqui decidimos pegar todas as palavras que iniciam os comentários extraídos, de modo a torná-las estados inicias no nosso automato
for word_begin in re.findall(r'\n\w+\b', comments): 
    word_begin = word_begin.replace('\n', '')
    first_words.append(word_begin)

# Removendo caracteres especiais para o texto ficar um pouquinho menos poluído
comments = comments.replace('\t',' ')
comments = comments.replace('“', ' ')
comments = comments.replace('”', ' ')
comments = comments.replace('"', ' ')

# Muitas pontuações estão coladas às palavras, e aqui descolamos a fim de serem consideradas palavras
for pontuation in pontuations:
    comments = comments.replace(pontuation, ' {0} '.format(pontuation))

# Para ficar mais emocionante, decidimos ignorar comentários muito pequenos
# Além disso, aqui já pegamos as palavras que encerram os comentários (exceto pontuações) a fim de torna-las estados finais
for line in comments.split('\n'):
    words = line.split(' ')
    if len(words) < 6:
        continue
    for word in reversed(words):
        if word == '' or word in pontuations:
            continue
        end_words.append(word)
        break

# Torna tudo uma linha só tirando as quebras de linha. Não é interessante pro algoritmo que pensamos que haja quebra de linhas, então simplesmente a removemos
comments = comments.replace('\n', ' ')

# Dados prontos para usar
comments = comments.split()
first_words = np.array(first_words)
end_words = np.array(end_words)

Aqui basicamente mapeamentos cada palavra X de modo que sabemos quantas vezes uma palavra Y apareceu seguida de X.

Exemplo:

"Eu gosto de Cachorro"

"Eu gosto de Gato"

Assim, contamos que "gosto" apareceu seguida de "Eu" 2 vezes, enquanto "Cachorro" e "Gato" apareceram seguidos de "de" uma vez cada

Será útil para estimar as probabilidades de chegar a alguma palavra a partir de outra

In [2]:
word_pairs = {}

for i, word in enumerate(comments[1:]):
    previous_word = comments[i-1].lower()
    word = word.lower()
    
    if previous_word not in word_pairs:
        word_pairs[previous_word] = {}
    if word in word_pairs[previous_word]:
        word_pairs[previous_word][word] += 1
    else:    
        word_pairs[previous_word][word] = 1

Agora basta construírmos nosso modelo:

Da forma que fizemos, há um parâmetro para escolhermos o número de palavras. Se esse parâmetro não for preenchido (ou seja, valor default fica como -1), ele vai continuar gerando palavras até chegar no estado final, sendo que usamos uma condição extra para que as frases ficassem um pouco mais longas, que é caso chegue num estado final ele tem 50% de chance de parar.

In [3]:
def markov_chain(first_word, total_words=-1):
    sentence = first_word.capitalize()
    current_word = first_word.lower()
    if total_words <= -1:
        total_words = 50
        stop_word = True
    else:
        stop_word = False
    
    for i in range(total_words):
        current_word_pairs = word_pairs[current_word]
        total = sum(current_word_pairs.values())
        if(total == 0): continue
        
        next_word = []
        weight = []
        for key, word in current_word_pairs.items():
            next_word.append(key)
            weight.append(word/total)
        next_word = np.array(next_word)
        weight = np.array(weight)
        
        chosen_word = np.random.choice(next_word, p=weight).strip() # Esse strip é semelhante ao "trim()" em algumas linguagens. Basicamente remove espaços em branco nas extremidades de uma String
        if chosen_word in pontuations:
            sentence += chosen_word
        elif current_word in ['.', '?'] or chosen_word == 'i':
            sentence += " " + chosen_word.capitalize()
        else:
            sentence += " " + chosen_word
        if stop_word and chosen_word in end_words:
            if np.random.rand() > 0.5: 
                break
        current_word = chosen_word
    return sentence

Podemos obter alguns resultados inserindo algumas palavras que servem de estados iniciais

In [4]:
random_first_words = np.random.choice(first_words, (10,))
random_first_words

for random_first_word in random_first_words:
    display(markov_chain(random_first_word))

'Uma alternativa fora ótimo'

'Feliz!!! parabéns ótimo'

'Melhor benefício bom recomendo'

'Continuem. Que, e áudio incrível os ficam. Forro fundo etc'

'A. Sei estrelas'

'Produto ótima amei'

'Produto parabéns produto muito mesmo essa um, tamanho'

'Material som. Bom'

'Otimo bonito'

'Exatamente anunciado'